# **Setup**

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path = '/content/drive/My Drive/Data Science/CP1/Data'

# **Merging the CSV files**

In [ ]:
dfs = []
csv_files = sorted([f for f in os.listdir(data_path) if f.endswith('.csv')])

for file in csv_files:
  area_name = file.replace(".csv", "")
  file_path = os.path.join(data_path, file)

  df = pd.read_csv(file_path)
  df['Area'] = area_name #adding a column with the area name
  dfs.append(df) #append the data frames

#merge files
merged_df = pd.concat(dfs, ignore_index=True, sort=True)
merged_df.tail()

In [ ]:
#save the file
output_path = os.path.join(data_path, 'merged_data.csv')
merged_df.to_csv(output_path, index=False)


# **Data Cleaning**

In [4]:
#Load the file
load_path = os.path.join(data_path, 'merged_data.csv')
df = pd.read_csv(load_path)

df.head()

,Rent_Type,Title,Location,Bedrooms,Bathrooms,Rent,Availability_Month,Gender,Property_URL,Area,Year
0,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,5.0,13000.0,April,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
1,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,4.0,65000.0,January,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
2,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,4.0,38000.0,March,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2025
3,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",3.0,4.0,28000.0,September,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
4,Room,Office House Rent,"Nikunja 1, Khilkhet",1.0,1.0,11000.0,April,Not specified,https://www.thetolet.com/bd/property-post/dhak...,Khilkhet,2025


In [5]:
#check how many title has family
df['Title'].str.contains('family', case=False).sum()


924

In [6]:
#delete title and links containing family
df = df[~df['Title'].str.contains('family', case=False, na=False) & ~df['Property_URL'].str.contains('family', case=False, na=False)]

In [7]:
#delete title and links containing office
df = df[~df['Title'].str.contains('office', case=False, na=False) & ~df['Property_URL'].str.contains('office', case=False, na=False)]

In [8]:
df.shape

(18990, 11)

In [9]:
#checking the year column
df['Year'].value_counts().sort_index()

,count
Year,
2022,217
2023,2643
2024,6546
2025,9582
2026,2


In [10]:
#Since 2022 has very low count, we'll remove it
df = df[~df['Year'].isin([2022,2026])]
print(df['Year'].value_counts().sort_index())
print(df.shape)

Year
2023    2643
2024    6546
2025    9582
Name: count, dtype: int64
(18771, 11)


In [11]:
#checking for any null values
df.isnull().sum()

,0
Rent_Type,1
Title,1
Location,1
Bedrooms,484
Bathrooms,1
Rent,1
Availability_Month,1
Gender,1
Property_URL,1
Area,1


In [12]:
col_clean = [col for col in df.columns if col != 'Bedrooms']
df = df.dropna(subset=col_clean)

In [13]:
#Investigate the cause and fill it up with relevant data

null_bedrooms_subset = df[df['Bedrooms'].isnull()]
print(null_bedrooms_subset[['Title','Bathrooms']].value_counts())


Title                Bathrooms
Sublet Room Rent     1.0          339
Bachelor Seat Rent   1.0           76
Bachelor Room Rent   1.0           33
Sublet Seat Rent     1.0           27
Hostel Seat Rent     1.0            4
Bachelor House Rent  1.0            1
Bachelor Seat Rent   2.0            1
Sublet House Rent    1.0            1
Sublet Room Rent     2.0            1
Name: count, dtype: int64


In [14]:
condition = (df['Bedrooms'].isnull()) & (df['Bathrooms'] == 1) & (df['Title'].str.contains('Seat', case=False, na=False))
df.loc[condition, 'Bedrooms'] = 1

In [15]:
condition = (df['Bedrooms'].isnull()) & (df['Bathrooms'] == 1) & (df['Title'].str.contains('Sublet', case=False, na=False))
df.loc[condition, 'Bedrooms'] = 1

In [16]:
condition = (df['Bedrooms'].isnull()) & (df['Bathrooms'] == 1) & (df['Title'].str.contains('Bachelor', case=False, na=False))
df.loc[condition, 'Bedrooms'] = 1

In [17]:
null_bedrooms = df[df['Bedrooms'].isnull()]
null_bedrooms

,Rent_Type,Title,Location,Bedrooms,Bathrooms,Rent,Availability_Month,Gender,Property_URL,Area,Year
461,Room,Sublet Room Rent,"Mohammadpur, Dhaka",NaN,2.0,12000.0,June,Anyone,https://www.thetolet.com/bd/property-post/dhak...,Mohammadpur,2025
561,Seat,Bachelor Seat Rent,"Mohammadpur, Dhaka",NaN,2.0,8500.0,July,Anyone,https://www.thetolet.com/bd/property-post/dhak...,Mohammadpur,2025


In [18]:
df['Bedrooms'] = df['Bedrooms'].fillna(2)

In [20]:
df.head()

,Rent_Type,Title,Location,Bedrooms,Bathrooms,Rent,Availability_Month,Gender,Property_URL,Area,Year
0,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,5.0,13000.0,April,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
1,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,4.0,65000.0,January,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
2,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",4.0,4.0,38000.0,March,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2025
3,Flat,Bachelor House Rent,"Bashundhara R.A, Vatara",3.0,4.0,28000.0,September,Male,https://www.thetolet.com/bd/property-post/dhak...,Vatara,2024
5,Flat,Bachelor Flat Rent,"Uttara, Dhaka",3.0,4.0,25000.0,October,Male,https://www.thetolet.com/bd/property-post/dhak...,Uttara,2024


In [19]:
df.isnull().sum()

,0
Rent_Type,0
Title,0
Location,0
Bedrooms,0
Bathrooms,0
Rent,0
Availability_Month,0
Gender,0
Property_URL,0
Area,0


In [21]:
(df['Rent'] == 0).sum()

np.int64(30)

In [22]:
df = df[df['Rent'] != 0]

In [23]:
df.nsmallest(5, 'Rent')

,Rent_Type,Title,Location,Bedrooms,Bathrooms,Rent,Availability_Month,Gender,Property_URL,Area,Year
20558,Seat,Bachelor Seat Rent,"Moghbazar, Ramna",2.0,1.0,1.0,September,Male,https://www.thetolet.com/bd/property-post/dhak...,Ramna,2023
20557,Seat,Bachelor Seat Rent,"Shahjahanpur, Dhaka",1.0,1.0,200.0,June,Male,https://www.thetolet.com/bd/property-post/dhak...,Shahjahanpur,2025
20556,Seat,Bachelor Seat Rent,"Agargaon, Sher-E-Bangla Nagar",1.0,1.0,300.0,August,Male,https://www.thetolet.com/bd/property-post/dhak...,Sher-E-Bangla Nagar,2025
20555,Seat,Bachelor Seat Rent,"Kajla, Jatrabari",1.0,1.0,600.0,October,Male,https://www.thetolet.com/bd/property-post/dhak...,Jatrabari,2023
822,Flat,Bachelor House Rent,"Shonir Akhra, Jatrabari",2.0,2.0,1000.0,March,Male,https://www.thetolet.com/bd/property-post/dhak...,Jatrabari,2024


In [24]:
df = df[df['Rent'] >= 1000 ]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18736 entries, 0 to 20554
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rent_Type           18736 non-null  object 
 1   Title               18736 non-null  object 
 2   Location            18736 non-null  object 
 3   Bedrooms            18736 non-null  float64
 4   Bathrooms           18736 non-null  float64
 5   Rent                18736 non-null  float64
 6   Availability_Month  18736 non-null  object 
 7   Gender              18736 non-null  object 
 8   Property_URL        18736 non-null  object 
 9   Area                18736 non-null  object 
 10  Year                18736 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 1.7+ MB


In [26]:
df.describe()

,Bedrooms,Bathrooms,Rent,Year
count,18736.000000,18736.000000,18736.000000,18736.000000
mean,1.021830,1.009874,4734.247865,2024.371904
std,0.216243,0.117162,3073.520245,0.715775
min,1.000000,1.000000,1000.000000,2023.000000
25%,1.000000,1.000000,2600.000000,2024.000000
50%,1.000000,1.000000,3500.000000,2025.000000
75%,1.000000,1.000000,6200.000000,2025.000000
max,10.000000,5.000000,65000.000000,2025.000000


In [27]:
#save the dataframe to csv file

output_path = os.path.join(data_path, 'bachelor_rentals_cleaned.csv')
df.to_csv(output_path, index=False)